In [10]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

# 参数设置

# 训练参数 
除了路径 其他不要变

In [11]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = r"E:\TECO_Works\Conference\ISWC2022\Run_logs"              
args.freq_save_path   = r"E:\TECO_Works\Conference\ISWC2022\Freq_data"
args.window_save_path = r"E:\TECO_Works\Conference\ISWC2022\Sliding_window"
args.root_path        = r"E:\datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 7
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

## 数据参数



In [12]:
args.seed                             = 1


args.data_name                        =  "wisdm"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6


args.difference       = False 
args.filtering        =  False
args.magnitude        =  False
args.weighted_sampler = False




args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]


if args.difference:
    args.c_in = args.c_in*2

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



## 模型参数

In [13]:
args.filter_scaling_factor = 0.25
args.model_type              = "deepconvlstm"#"deepconvlstm"#"sahar" #"deepconvlstm"
exp = Exp(args)
exp.model

Use GPU: cuda:0
Build the DeepConvLSTM model!
Done!
Parameter : 14678
Set the seed as :  1


model_builder(
  (model): DeepConvLSTM(
    (conv_blocks): ModuleList(
      (0): ConvBlock(
        (conv1): Conv2d(1, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
      (1): ConvBlock(
        (conv1): Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
    )
    (lstm_layers): ModuleList(
      (0): LSTM(48, 32, batch_first=True)
    )
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [14]:
# cross_channel_interaction_type = "attn",    # attn  transformer  identity
# cross_channel_aggregation_type = "filter",  # filter  naive  FC  "SFCC", "SFCF"
# temporal_info_interaction_type = "gru",     # gru  lstm  attn  transformer  identity  conv
# temporal_info_aggregation_type = "FC",      # naive  filter  FC  tnaive

# 实验

In [15]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

Use GPU: cuda:0
Build the TinyHAR model!
Done!
Parameter : 16184
Set the seed as :  1


In [16]:
exp.model

model_builder(
  (model): TinyHAR_Model(
    (layers_conv): ModuleList(
      (0): Sequential(
        (0): Conv2d(1, 16, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (3): Sequential(
        (0): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (channel_interaction): SelfAttention_intera

In [ ]:
exp.train()

 ----------------------- load all the data -------------------
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 10 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.0001465  0.000181   0.00097182 0.00123001 0.0004852  0.00060716]
Train data number :  17901
The number of classes is :  6
The input_length  is :  100
The channel_in is :  3
Validation data number :  1989
Test data number :  9947
================ Build the model ================ 
Build the TinyHAR model!
Epoch: 1 cost time: 5.407215118408203
VALI: Epoch: 1, Steps: 70 | Train Loss: 1.1796709  Vali Loss: 0.6614299 Vali Accuracy: 0.7415787  Vali weighted F1: 0.6453708  Vali macro F1 0.4268904 
Validation loss decreased (inf --> 0.661430).  Saving model

Epoch: 28 cost time: 5.4304633140563965
VALI: Epoch: 28, Steps: 70 | Train Loss: 0.0654643  Vali Loss: 0.1680940 Vali Accuracy: 0.9532428  Vali weighted F1: 0.9525918  Vali macro F1 0.9397371 
EarlyStopping counter: 7 out of 15
Learning rate adjusting counter: 7 out of 7
Updating learning rate to 0.0001
Epoch: 29 cost time: 5.4297034740448
VALI: Epoch: 29, Steps: 70 | Train Loss: 0.0554008  Vali Loss: 0.1478626 Vali Accuracy: 0.9587733  Vali weighted F1: 0.9585681  Vali macro F1 0.9460909 
EarlyStopping counter: 8 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 30 cost time: 5.4496448040008545
VALI: Epoch: 30, Steps: 70 | Train Loss: 0.0479540  Vali Loss: 0.1459777 Vali Accuracy: 0.9602815  Vali weighted F1: 0.9600930  Vali macro F1 0.9486720 
EarlyStopping counter: 9 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 31 cost time: 5.432219982147217
VALI: Epoch: 31, Steps: 70 | Train Loss: 0.0479216  Vali Loss: 0.1451909 Vali Accuracy: 0.9592760  Vali weighted F1

VALI: Epoch: 9, Steps: 68 | Train Loss: 0.1628179  Vali Loss: 0.1817109 Vali Accuracy: 0.9388284  Vali weighted F1: 0.9388605  Vali macro F1 0.9158277 
new best score!!!!
Validation loss decreased (0.224033 --> 0.181711).  Saving model ...
new best score!!!!
Epoch: 10 cost time: 5.358184814453125
VALI: Epoch: 10, Steps: 68 | Train Loss: 0.1510255  Vali Loss: 0.1917924 Vali Accuracy: 0.9315708  Vali weighted F1: 0.9324620  Vali macro F1 0.9074216 
EarlyStopping counter: 1 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 11 cost time: 5.269776105880737
VALI: Epoch: 11, Steps: 68 | Train Loss: 0.1372602  Vali Loss: 0.1815125 Vali Accuracy: 0.9403836  Vali weighted F1: 0.9401655  Vali macro F1 0.9208378 
new best score!!!!
Validation loss decreased (0.181711 --> 0.181513).  Saving model ...
new best score!!!!
Epoch: 12 cost time: 5.278193712234497
VALI: Epoch: 12, Steps: 68 | Train Loss: 0.1301128  Vali Loss: 0.1641634 Vali Accuracy: 0.9486781  Vali weighted F1: 0.9490143  Vali

Epoch: 39 cost time: 5.277698755264282
VALI: Epoch: 39, Steps: 68 | Train Loss: 0.0250925  Vali Loss: 0.1365911 Vali Accuracy: 0.9569725  Vali weighted F1: 0.9572385  Vali macro F1 0.9437699 
EarlyStopping counter: 12 out of 15
Learning rate adjusting counter: 5 out of 7
Epoch: 40 cost time: 5.28121280670166
VALI: Epoch: 40, Steps: 68 | Train Loss: 0.0269953  Vali Loss: 0.1372060 Vali Accuracy: 0.9606013  Vali weighted F1: 0.9608624  Vali macro F1 0.9477027 
EarlyStopping counter: 13 out of 15
Learning rate adjusting counter: 6 out of 7
Epoch: 41 cost time: 5.27119255065918
VALI: Epoch: 41, Steps: 68 | Train Loss: 0.0259208  Vali Loss: 0.1394551 Vali Accuracy: 0.9631934  Vali weighted F1: 0.9632014  Vali macro F1 0.9519009 
EarlyStopping counter: 14 out of 15
Learning rate adjusting counter: 7 out of 7
Updating learning rate to 1e-05
Epoch: 42 cost time: 5.2792017459869385
VALI: Epoch: 42, Steps: 68 | Train Loss: 0.0241736  Vali Loss: 0.1375977 Vali Accuracy: 0.9621566  Vali weighted F

Epoch: 24 cost time: 5.383183002471924
VALI: Epoch: 24, Steps: 70 | Train Loss: 0.0916541  Vali Loss: 0.1104492 Vali Accuracy: 0.9659033  Vali weighted F1: 0.9660139  Vali macro F1 0.9518973 
new best score!!!!
Validation loss decreased (0.113542 --> 0.110449).  Saving model ...
new best score!!!!
Epoch: 25 cost time: 5.374215602874756
VALI: Epoch: 25, Steps: 70 | Train Loss: 0.0667444  Vali Loss: 0.1347332 Vali Accuracy: 0.9572519  Vali weighted F1: 0.9571472  Vali macro F1 0.9396316 
EarlyStopping counter: 1 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 26 cost time: 5.3802170753479
VALI: Epoch: 26, Steps: 70 | Train Loss: 0.0675942  Vali Loss: 0.1172581 Vali Accuracy: 0.9597964  Vali weighted F1: 0.9604623  Vali macro F1 0.9435571 
EarlyStopping counter: 2 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 27 cost time: 5.366213321685791
VALI: Epoch: 27, Steps: 70 | Train Loss: 0.0763665  Vali Loss: 0.1207626 Vali Accuracy: 0.9572519  Vali weighted F1: 0.957

Epoch: 54 cost time: 5.3882105350494385
VALI: Epoch: 54, Steps: 70 | Train Loss: 0.0382486  Vali Loss: 0.1128345 Vali Accuracy: 0.9638677  Vali weighted F1: 0.9639792  Vali macro F1 0.9474610 
EarlyStopping counter: 6 out of 15
Learning rate adjusting counter: 6 out of 7
Epoch: 55 cost time: 5.369214296340942
VALI: Epoch: 55, Steps: 70 | Train Loss: 0.0347962  Vali Loss: 0.1130341 Vali Accuracy: 0.9623410  Vali weighted F1: 0.9624588  Vali macro F1 0.9456785 
EarlyStopping counter: 7 out of 15
Learning rate adjusting counter: 7 out of 7
Updating learning rate to 1.0000000000000002e-06
Epoch: 56 cost time: 5.378523349761963
VALI: Epoch: 56, Steps: 70 | Train Loss: 0.0369329  Vali Loss: 0.1104155 Vali Accuracy: 0.9638677  Vali weighted F1: 0.9641133  Vali macro F1 0.9476747 
EarlyStopping counter: 8 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 57 cost time: 5.43044638633728
VALI: Epoch: 57, Steps: 70 | Train Loss: 0.0354214  Vali Loss: 0.1107417 Vali Accuracy: 0.9638677  

Epoch: 18 cost time: 5.348210334777832
VALI: Epoch: 18, Steps: 69 | Train Loss: 0.0971116  Vali Loss: 0.1326859 Vali Accuracy: 0.9572606  Vali weighted F1: 0.9566559  Vali macro F1 0.9402472 
EarlyStopping counter: 2 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 19 cost time: 5.384218692779541
VALI: Epoch: 19, Steps: 69 | Train Loss: 0.0865770  Vali Loss: 0.1436029 Vali Accuracy: 0.9536560  Vali weighted F1: 0.9525131  Vali macro F1 0.9341060 
EarlyStopping counter: 3 out of 15
Learning rate adjusting counter: 3 out of 7
Epoch: 20 cost time: 5.3672144412994385
VALI: Epoch: 20, Steps: 69 | Train Loss: 0.0864747  Vali Loss: 0.1324304 Vali Accuracy: 0.9552008  Vali weighted F1: 0.9547277  Vali macro F1 0.9352481 
EarlyStopping counter: 4 out of 15
Learning rate adjusting counter: 4 out of 7
Epoch: 21 cost time: 5.330205917358398
VALI: Epoch: 21, Steps: 69 | Train Loss: 0.0756051  Vali Loss: 0.1316108 Vali Accuracy: 0.9557158  Vali weighted F1: 0.9548460  Vali macro F1 0.938

In [ ]:
dataset = data_dict[args.data_name](args)

In [ ]:
import numpy as np

In [ ]:
start_index = dataset.train_slidingwindows[0][1]
end_index   = dataset.train_slidingwindows[0][2]
sample_x_1    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

start_index = dataset.train_slidingwindows[100][1]
end_index   = dataset.train_slidingwindows[100][2]
sample_x_2    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

temp_1 = np.expand_dims(sample_x_1,0)
temp_2 = np.expand_dims(sample_x_2,0)
combined_x = np.concatenate([temp_1,temp_2],axis=0)
combined_x = np.expand_dims(combined_x,1)
combined_x = torch.tensor(combined_x).double().to(exp.device)
print(combined_x.shape)

In [ ]:
out = exp.model(combined_x)
out = out.cpu().detach().numpy()

In [ ]:
out.shape

In [ ]:
weight = exp.model.wave_conv.wavelet_conv.weight.cpu().detach().numpy()

In [ ]:
for n,p in exp.model.named_parameters():
    print(n)

In [ ]:
import matplotlib.pyplot as plt
index1 = 0
index2 = 0

plt.figure()
plt.plot(out[0,0,:,index2])
plt.plot(out[0,1,:,index2])
plt.plot(out[0,2,:,index2])
plt.plot(out[0,3,:,index2])

In [ ]:
# 如果我们设置为 
# args.wavelet_filtering_learnable      = True
# exp = Exp(args)